## Power iteration

In [3]:
import numpy as np

def power_iteration(A, tol=1e-7):
    b = np.random.rand(A.shape[1])
    b_prev = None
    n_iters = 0
    while n_iters == 0 or np.linalg.norm(b - b_prev) > tol:
        b_prev = b.copy()
        b = np.dot(A, b)
        b /= np.linalg.norm(b)
        n_iters += 1
    return np.dot(A, b), b, n_iters

In [9]:
def main():
    A = np.array([[0.5, 0.4], [0.2, 0.8]])
    ab, b, n_iters = power_iteration(A)
    
    eig1 = ab[0] / b[0]
    eig2 = ab[1] / b[1]
    assert np.abs((eig1 - eig2) / eig2) < 1e-5
    
    b /= b[1]
    
    print(eig1, b, n_iters)

main()

0.9701561772632724 [0.85078112 1.        ] 14


## word2vec

In [2]:
import sys
sys.path.insert(0, '../a2')

import numpy as np
from scipy.special import softmax

### naive

In [13]:
n_vocab = 100
n_embed = 50
outsideWordIdx = np.random.randint(n_vocab)

In [14]:
outsideVectors = np.random.randn(n_vocab, n_embed)
centerWordVec = np.random.randn(n_embed)

In [15]:
z = outsideVectors @ centerWordVec; z.shape

(100,)

In [16]:
y_hat = softmax(z)

In [17]:
loss = -np.log(y_hat[outsideWordIdx]); loss

20.219017897408527

In [18]:
y_hat_mod = y_hat.copy()
y_hat_mod[outsideWordIdx] -= 1

In [19]:
gradCenterVec = np.dot(outsideVectors.T, y_hat_mod); gradCenterVec.shape

(50,)

In [20]:
gradOutsideVecs = np.outer(y_hat_mod, centerWordVec); gradOutsideVecs.shape

(100, 50)

### negative sampling

In [21]:
from word2vec import sigmoid

In [22]:
k = 10

In [28]:
negSampleWordIndices = np.random.choice(n_vocab, k).tolist(); negSampleWordIndices

[36, 60, 95, 97, 30, 26, 74, 20, 91, 91]

In [29]:
indices = [outsideWordIdx] + negSampleWordIndices

In [38]:
outsideWordIdx, indices

(34, [34, 36, 60, 95, 97, 30, 26, 74, 20, 91, 91])

In [32]:
sampleOutsideVectors = outsideVectors[indices]
z = np.dot(sampleOutsideVectors, centerWordVec)
z[1:] = -z[1:]

In [33]:
probs = sigmoid(z)
loss = -np.sum(np.log(probs))

In [34]:
cprobs = 1 - probs
cprobs[0] = -cprobs[0]

In [35]:
sampleOutsideVectors.shape, cprobs.shape

((11, 50), (11,))

In [37]:
grad = np.outer(cprobs, centerWordVec); grad.shape

(11, 50)